### Генерация поэзии с помощью нейронных сетей: шаг 1
##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

Ваша основная задача: научиться генерироват стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.

In [29]:
# do not change the code in the block below
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt
# __________end of block__________

In [2]:
# do not change the code in the block below
# __________start of block__________
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))
# __________end of block__________

cpu device is available


In [7]:
# Загрузка файла с помощью curl
!curl -o onegin.txt https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  256k  100  256k    0     0   389k      0 --:--:-- --:--:-- --:--:--  389k


#### 1. Загрузка данных.

In [8]:
# do not change the code in the block below
# __________start of block__________
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt%%!
    
with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()
    
text = "".join([x.replace('\t\t', '').lower() for x in text])
# __________end of block__________

zsh:1: command not found: wget


In [13]:
text[-100:]

'а\nбокала полного вина,\nкто не дочел ее романа\nи вдруг умел расстаться с ним,\nкак я с онегиным моим.\n'

#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим токен `<sos>`.

In [14]:
# do not change the code in the block below
# __________start of block__________
tokens = sorted(set(text.lower())) + ['<sos>']
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятие №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [20]:
# do not change the code in the block below
# __________start of block__________
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx['<sos>']

def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))
# __________end of block__________    

Пример батча:

In [27]:
a = next(generate_chunk())
a


array([[83, 53, 62, ..., 63, 53, 66],
       [83, 45, 66, ..., 55, 53,  1],
       [83, 57, 59, ..., 52, 53, 57],
       ...,
       [83,  1, 59, ...,  7,  0, 66],
       [83, 56, 45, ..., 61, 63, 58],
       [83, 72, 50, ..., 76, 63, 73]])

In [30]:
class CharRNN(nn.Module):
    def __init__(self, num_tokens, hidden_size, n_layers=2, dropout=0.3):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(num_tokens, hidden_size)

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True, dropout=dropout)

        self.fc = nn.Linear(hidden_size, num_tokens)
    
    def forward(self, x, hidden):

        x = self.embedding(x)

        output, hidden = self.gru(x, hidden)

        output = self.fc(output)

        return output, hidden
    
    def init_hidden(self, batch_size):

        return torch.zeros(self.n_layers, batch_size, self.hidden_size)

In [35]:
hidden_size = 256
n_layers = 2
num_tokens = len(tokens)
dropout = 0.3

model = CharRNN(num_tokens, hidden_size, n_layers, dropout=dropout)
optimizer = optim.Adam(model.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss()

def train(model, num_epochs=100, batch_size=256, seq_length=100):
    model.train()
    losses = []
    for epoch in range(num_epochs):
        hidden = model.init_hidden(batch_size)
        for i, batch in enumerate(generate_chunk()):
            batch = torch.tensor(batch, dtype=torch.long)
            inputs, targets = batch[:, :-1], batch[:, 1:].reshape(-1)

            optimizer.zero_grad()
            output, hidden = model(inputs, hidden)
            loss = criterion(output.view(-1, num_tokens), targets)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())
            if i % 100 == 0:
                print(f'Epoch: {epoch}, Step: {i}, Loss: {loss.item()}')
    return losses

losses = train(model)


Epoch: 0, Step: 0, Loss: 4.437472820281982
Epoch: 1, Step: 0, Loss: 4.186212539672852
Epoch: 2, Step: 0, Loss: 3.7909586429595947
Epoch: 3, Step: 0, Loss: 3.4447176456451416
Epoch: 4, Step: 0, Loss: 3.3681161403656006
Epoch: 5, Step: 0, Loss: 3.327181816101074
Epoch: 6, Step: 0, Loss: 3.266806125640869
Epoch: 7, Step: 0, Loss: 3.241899013519287
Epoch: 8, Step: 0, Loss: 3.2163071632385254
Epoch: 9, Step: 0, Loss: 3.1585259437561035
Epoch: 10, Step: 0, Loss: 3.114703416824341
Epoch: 11, Step: 0, Loss: 3.0640954971313477
Epoch: 12, Step: 0, Loss: 3.0021677017211914
Epoch: 13, Step: 0, Loss: 2.9576408863067627
Epoch: 14, Step: 0, Loss: 2.9278857707977295
Epoch: 15, Step: 0, Loss: 2.89848256111145
Epoch: 16, Step: 0, Loss: 2.858078718185425
Epoch: 17, Step: 0, Loss: 2.8101871013641357
Epoch: 18, Step: 0, Loss: 2.7992825508117676
Epoch: 19, Step: 0, Loss: 2.7694191932678223
Epoch: 20, Step: 0, Loss: 2.7432570457458496
Epoch: 21, Step: 0, Loss: 2.726238489151001
Epoch: 22, Step: 0, Loss: 2.71

Шаблон функции `generate_sample` также доступен ниже. Вы можете как дозаполнить его, так и написать свою собственную функцию с нуля. Не забывайте, что все примеры в обучающей выборке начинались с токена `<sos>`.

In [36]:
def generate_sample(char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device):
    if seed_phrase is not None:
        x_sequence = [token_to_idx['<sos>']] + [token_to_idx[token] for token in seed_phrase]
    else: 
        x_sequence = [token_to_idx['<sos>']]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)
    hidden = char_rnn.init_hidden(1)

    generated_text = seed_phrase or ""
    for _ in range(max_length - len(generated_text)):
        output, hidden = char_rnn(x_sequence, hidden)
        output = output[:, -1, :] / temperature
        probabilities = torch.softmax(output, dim=1).data.cpu().numpy().ravel()
        next_token = np.random.choice(num_tokens, p=probabilities)
        
        x_sequence = torch.cat([x_sequence, torch.tensor([[next_token]], device=device)], dim=1)
        generated_text += idx_to_token[next_token]
    return generated_text.replace('<sos>', '')

# Генерация примеров
seed_phrase = ' мой дядя самых честных правил'
generated_phrases = [
    generate_sample(
        model,
        seed_phrase,
        max_length=500,
        temperature=0.8  # Экспериментируйте с температурой для лучшего качества
    )
    for _ in range(10)
]

Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [37]:
print(generate_sample(model, ' мой дядя самых честных правил', max_length=500, temperature=0.8))

 мой дядя самых честных правились,
а стра збостные перосювичной
остнею влов,
то токноворого поры, на взоренье одна был мовес кут никоньта
прожие сердце шлег одный праровует,
не пойней нески древала
их кашно перезнильки белясь негий черская,
стах нашет обыла ей бора
посел и, полчили,
свою ильмо, свезы,
лыва, зны двура жисненсосела раздунным…
ывен траний назверего.
во под мечтво нас попрелпять,
предлечал будно встердет она уж кати метел;
и ес их надрушах оная прод,
в седельный ж нердет ее млень,
н


### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [24]:
seed_phrase = ' мой дядя самых честных правил'

In [38]:
# do not change the code in the block below
# __________start of block__________

import json
if 'generated_phrases' not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all([x in set(tokens) for x in set(list(phrase))]), 'Unknown tokens detected, check your submission!'
    

submission_dict = {
    'token_to_idx': token_to_idx,
    'generated_phrases': generated_phrases
}

with open('submission_dict.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
print('File saved to `submission_dict.json`')
# __________end of block__________

File saved to `submission_dict.json`


На этом задание завершено. Поздравляем!